#Fine-Tuning OpenAI Models

Copyright 2024 Denis Rothman

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.Download and prepare the data   
2.Fine-tune a model   
3.Run a fine-tuned model

# Installing the environment


In [ ]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)
#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [ ]:
try:
  import openai
except:
  !pip install openai==1.42.0
  import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install jsonlines==4.0.0

In [ ]:
!pip install datasets==2.20.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 require

Listing the installed packages

In [ ]:
import subprocess

# Run pip list and capture the output
result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE, text=True)

# Split the output into lines and count them
package_list = result.stdout.split('\n')

# Adjust count for headers or empty lines
package_count = len([line for line in package_list if line.strip() != '']) - 2

print(f"Number of installed packages: {package_count}")

Number of installed packages: 513


In [ ]:
import subprocess

# Run pip list and capture the output
result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE, text=True)

# Print the output
print(result.stdout)

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.8.26.0.31.57
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread              

counting the number of packages

# 1.Preparing the dataset for fine-tuning

## 1.1.Downloading and displaying the dataset

In [ ]:
# Import required libraries
from datasets import load_dataset
import pandas as pd

# Load the SciQ dataset from HuggingFace
dataset_view = load_dataset("sciq", split="train")

# Filter the dataset to include only questions with support and correct answer
filtered_dataset = dataset_view.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")


# Print the number of questions with support
print("Number of questions with support: ", len(filtered_dataset))

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11679 [00:00<?, ? examples/s]

Number of questions with support:  10481


In [ ]:
# Convert the filtered dataset to a pandas DataFrame
df_view = pd.DataFrame(filtered_dataset)

# Columns to drop
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']

# Dropping the columns from the DataFrame
df_view = df_view.drop(columns=columns_to_drop)

# Display the DataFrame
df_view.head()

,question,correct_answer,support
0,What type of organism is commonly used in prep...,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


## 1.2A Streaming the output to JSON


In [ ]:
import json
import pandas as pd

## 1.2. Preparing the dataset for fine-tuning

In [ ]:
import jsonlines
import pandas as pd
from datasets import load_dataset

# Load and clean the dataset as previously described
dataset = load_dataset("sciq", split="train")
filtered_dataset = dataset.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")

# Convert to DataFrame and clean
df = pd.DataFrame(filtered_dataset)
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']
df = df.drop(columns=columns_to_drop)

# Prepare the data items for JSON lines file
items = []
for idx, row in df.iterrows():
    detailed_answer = row['correct_answer'] + " Explanation: " + row['support']
    items.append({
        "messages": [
            {"role": "system", "content": "Given a science question, provide the correct answer with a detailed explanation."},
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": detailed_answer}
        ]
    })

# Write to JSON lines file
with jsonlines.open('/content/QA_prompts_and_completions.json', 'w') as writer:
    writer.write_all(items)

### Visualizing the JSON file

In [ ]:
dfile="/content/QA_prompts_and_completions.json"

In [ ]:
import pandas as pd

# Load the data
df = pd.read_json(dfile, lines=True)
df

,messages
0,"[{'role': 'system', 'content': 'Given a scienc..."
1,"[{'role': 'system', 'content': 'Given a scienc..."
2,"[{'role': 'system', 'content': 'Given a scienc..."
3,"[{'role': 'system', 'content': 'Given a scienc..."
4,"[{'role': 'system', 'content': 'Given a scienc..."
...,...
10476,"[{'role': 'system', 'content': 'Given a scienc..."
10477,"[{'role': 'system', 'content': 'Given a scienc..."
10478,"[{'role': 'system', 'content': 'Given a scienc..."
10479,"[{'role': 'system', 'content': 'Given a scienc..."


# 2.Fine-tuning the model



In [ ]:
from openai import OpenAI
import jsonlines
client = OpenAI()
# Uploading the training file

result_file = client.files.create(
  file=open("QA_prompts_and_completions.json", "rb"),
  purpose="fine-tune"
)

print(result_file)
param_training_file_name = result_file.id
print(param_training_file_name)

# Creating the fine-tuning job

ft_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-mini-2024-07-18"
)

# Printing the fine-tuning job
print(ft_job)

FileObject(id='file-EUPGmm1yAd3axrQ0pyoeAKuE', bytes=8062970, created_at=1725289249, filename='QA_prompts_and_completions.json', object='file', purpose='fine-tune', status='processed', status_details=None)
file-EUPGmm1yAd3axrQ0pyoeAKuE
FineTuningJob(id='ftjob-O1OEE7eEyFNJsO2Eu5otzWA8', created_at=1725289250, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-h2Kjmcir4wyGtqq1mJALLGIb', result_files=[], seed=1103096818, status='validating_files', trained_tokens=None, training_file='file-EUPGmm1yAd3axrQ0pyoeAKuE', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


## Monitoring the fine-tunes

In [ ]:
import pandas as pd
from openai import OpenAI
client = OpenAI()
# Assume client is already set up and authenticated
response = client.fine_tuning.jobs.list(limit=3) # increase to include your history

# Initialize lists to store the extracted data
job_ids = []
created_ats = []
statuses = []
models = []
training_files = []
error_messages = []
fine_tuned_models = []  # List to store the fine-tuned model names

# Iterate over the jobs in the response
for job in response.data:
    job_ids.append(job.id)
    created_ats.append(job.created_at)
    statuses.append(job.status)
    models.append(job.model)
    training_files.append(job.training_file)
    error_message = job.error.message if job.error else None
    error_messages.append(error_message)

    # Append the fine-tuned model name
    fine_tuned_model = job.fine_tuned_model if hasattr(job, 'fine_tuned_model') else None
    fine_tuned_models.append(fine_tuned_model)

# Create a DataFrame
df = pd.DataFrame({
    'Job ID': job_ids,
    'Created At': created_ats,
    'Status': statuses,
    'Model': models,
    'Training File': training_files,
    'Error Message': error_messages,
    'Fine-Tuned Model': fine_tuned_models  # Include the fine-tuned model names
})

# Convert timestamps to readable format
df['Created At'] = pd.to_datetime(df['Created At'], unit='s')
df = df.sort_values(by='Created At', ascending=False)

# Display the DataFrame
df

,Job ID,Created At,Status,Model,Training File,Error Message,Fine-Tuned Model
0,ftjob-O1OEE7eEyFNJsO2Eu5otzWA8,2024-09-02 15:00:50,running,gpt-4o-mini-2024-07-18,file-EUPGmm1yAd3axrQ0pyoeAKuE,None,None
1,ftjob-gQGiuvPMvSop0tzGaDn1NMql,2024-09-02 14:26:35,succeeded,gpt-4o-mini-2024-07-18,file-1OyEhi0D2b1kcL54JbQ3P1Pa,None,ft:gpt-4o-mini-2024-07-18:personal::A32qtJOo
2,ftjob-oVB0RAcwn3NEi4u0qOMeMZUF,2024-09-02 14:11:28,succeeded,gpt-4o-mini-2024-07-18,file-0dxQmL84uLME7ehnGIjQAxit,None,ft:gpt-4o-mini-2024-07-18:personal::A32VfYIz


### Make sure to obtain your fine-tune model here

If your OpenAI notifications are activated you should receive an email.

Otherwise run the "Monitoring the fine-tunes" cell above to check the status of your fine-tune job.

In [ ]:
import pandas as pd

generation=False  # until the current model is fine-tuned
# Attempt to find the first non-empty Fine-Tuned Model
non_empty_models = df[df['Fine-Tuned Model'].notna() & (df['Fine-Tuned Model'] != '')]

if not non_empty_models.empty:
    first_non_empty_model = non_empty_models['Fine-Tuned Model'].iloc[0]
    print("The latest fine-tuned model is:", first_non_empty_model)
    generation=True
else:
    first_non_empty_model='None'
    print("No fine-tuned models found.")

The latest fine-tuned model is: ft:gpt-4o-mini-2024-07-18:personal::A32VfYIz


*Note:* Only continue to Step 3, to use the fine-tuned model when your fine-tuned model is ready. If your OpenAI notifications is activiated, you will receive an email with the status of your fine-tunning job.

# 3.Using the fine-tuned OpenAI model

Note: The is a fine-tuning. As such, be patient!
Rune the `Monitoring the fine-tunes` cell and the f`irst_non_empty_model` cell from time to time.

If the fine-tunning succeeded and your model is ready, the name of your model will be `first_non_empty_model`

1.Go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.Check the metrics in the fine-tuning UI:
https://platform.openai.com/finetune/

3.Try the fined-tune model out in the cell below.

In [ ]:
# Define the prompt
prompt = "What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?"

*Note:* Only run the following cell if your fine-tune job has succeeded and a fined-tuned model is found in the *Monitoring the fine-tunes"* section of *2.Fine-tuning the model.*

In [ ]:
# Assume first_non_empty_model is defined above this snippet
if generation==True:
    response = client.chat.completions.create(
        model=first_non_empty_model,
        temperature=0.0,  # Adjust as needed for variability
        messages=[
            {"role": "system", "content": "Given a question, reply with a complete explanation for students."},
            {"role": "user", "content": prompt}
        ]
    )
else:
    print("Error: Model is None, cannot proceed with the API request.")

In [ ]:
if generation==True:
  print(response)

ChatCompletion(id='chatcmpl-A32pvH9wLvNsSRmB1sUjxOW4Z6Xr6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Coriolis effect Explanation: The Coriolis effect is a phenomenon that causes moving objects, such as air and water, to turn and twist in response to the rotation of the Earth. It is responsible for the rotation of large weather systems, such as hurricanes, and the direction of trade winds and ocean currents. In the Northern Hemisphere, the effect causes moving objects to turn to the right, while in the Southern Hemisphere, objects turn to the left. The Coriolis effect is proportional to the speed of the moving object and the strength of the Earth's rotation, and it is negligible for small-scale movements, such as water flowing in a sink.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725289251, model='ft:gpt-4o-mini-2024-07-18:personal::A32VfYIz', object='chat.completion', service_tier=None, 

In [ ]:
if (generation==True):
  # Access the response from the first choice
  response_text = response.choices[0].message.content
  # Print the response
  print(response_text)

Coriolis effect Explanation: The Coriolis effect is a phenomenon that causes moving objects, such as air and water, to turn and twist in response to the rotation of the Earth. It is responsible for the rotation of large weather systems, such as hurricanes, and the direction of trade winds and ocean currents. In the Northern Hemisphere, the effect causes moving objects to turn to the right, while in the Southern Hemisphere, objects turn to the left. The Coriolis effect is proportional to the speed of the moving object and the strength of the Earth's rotation, and it is negligible for small-scale movements, such as water flowing in a sink.


In [ ]:
import textwrap

if generation==True:
  wrapped_text = textwrap.fill(response_text.strip(), 60)
  print(wrapped_text)

Coriolis effect Explanation: The Coriolis effect is a
phenomenon that causes moving objects, such as air and
water, to turn and twist in response to the rotation of the
Earth. It is responsible for the rotation of large weather
systems, such as hurricanes, and the direction of trade
winds and ocean currents. In the Northern Hemisphere, the
effect causes moving objects to turn to the right, while in
the Southern Hemisphere, objects turn to the left. The
Coriolis effect is proportional to the speed of the moving
object and the strength of the Earth's rotation, and it is
negligible for small-scale movements, such as water flowing
in a sink.


[Consult OpenAI fine-tune documentation for more](https://platform.openai.com/docs/guides/fine-tuning)